In [63]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import planck_colormap
import scipy
from scipy import special

%matplotlib inline

In [64]:
cm = planck_colormap.colormap()

In [65]:
### for testing
### set up - params
nside = 16
lmax = 3*nside//2
npix = hp.nside2npix(nside)
### importing maps
cl = np.load('cls_PlanckPR2_TT_lowp_lensing_lensed.npy')
tmap = hp.read_map('commander_t_map_n16.fits', verbose=False)
noise_cov = hp.read_map('commander_noise_cov_n16.fits',verbose=False)
mask = hp.read_map('commander_mask_n16.fits', verbose=False)

In [66]:
class WienerFilter:
    def __init__(self, nside, mask=None):
        self.params = Parameters(nside)
        if mask is not None:
            self.mask = Mask(mask, nside)
        else:
            self.mask = Mask(np.ones(self.params.npix))
            
    def set_signal_cov(self, cl):
        self.signal_cov = SignalCov(cl, 3*self.params.nside//2, self.params.nside)

In [67]:
class Parameters:
    def __init__(self, nside):
        """Set base parameters"""
        self.nside = nside
        self.npix = hp.nside2npix(nside)
        
class Mask:
    """Instantiate mask, get good_pix and bad_pix and degrade with 0.9 criteria
    if nside_out is specified."""
    def __init__(self, mask, nside_out=None):
        msk = np.copy(mask)
        if nside_out is not None:
            msk = hp.ud_grade(msk, nside_out)
            msk[msk >= 0.9] = 1
            msk[msk < 0.9] = 0
        self.good_pix = msk == 1
        self.bad_pix = msk == 0
        self.mask = msk


class SignalCov:
    """Calculate signal covariance matrix"""
    def __init__(self, cl, lmax, nside):
        self.lmax = lmax
        self.cl = cl[:lmax+1]
        self.l = np.arange(lmax+1)
        self.nside = nside
        self.npix = hp.nside2npix(nside)
        self.get_signal_cov()
        
    def get_signal_cov(self):
        cosines = self.get_cosines(self.nside)
        unique, uniqinv = self.get_unique(cosines)
        def s(x):
            return self.two_point(x, self.l, self.cl, self.lmax)
        s = np.vectorize(s)
        s_unique = s(unique)
        s_matrix = s_unique[uniqinv].reshape(self.npix,self.npix)
        self.signal_cov = s_matrix
        
    def get_cosines(self, nside):
        vectors = np.array(hp.pix2vec(nside,np.arange(hp.nside2npix(nside)))).transpose()
        cosines = np.einsum('ik,jk', vectors, vectors)
        cosines[cosines>1] =1
        cosines[cosines<-1]=-1
        return cosines
    
    def get_unique(self, cosines):
        uniq, uniqindex, uniqinv, uniqcounts = \
        np.unique(cosines.flatten(),return_index=True, return_inverse=True,return_counts=True)
        return uniq, uniqinv
    
    def two_point(self, x, l, cl, lmax):
        return np.sum((2*l+1)*cl*scipy.special.lpn(lmax,x)[0])/(4*np.pi)  